In [8]:
!pip install transformers konlpy

     |████████████████████████████████| 19.4MB 168kB/s 
     |████████████████████████████████| 460kB 49.3MB/s 
     |████████████████████████████████| 92kB 12.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [5]:
from transformers import BertConfig , BertForSequenceClassification
from transformers import BertTokenizer
import torch
model = BertForSequenceClassification.from_pretrained('beomi/kcbert-large')
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-large')

Some weights of the model checkpoint at beomi/kcbert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initial

In [6]:
model.load_state_dict(torch.load('kcbert_epoch10.pt'))

<All keys matched successfully>

In [9]:
from konlpy.tag import *
komoran = Komoran()

In [26]:
import numpy as np
def softmax(a):
  c = np.max(a)
  exp_a = np.exp(a-c)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a
  return y

In [10]:
def split_token_get(text,tokenizer,MAX):
  split_tokens = []
  for token in komoran.morphs(text):
    for sub_token in tokenizer.tokenize(token):
      split_tokens.append(sub_token)
  if len(split_tokens) > MAX-2:
    split_tokens = split_tokens[:(MAX-2)]
  split_tokens = ["[CLS]"]+ split_tokens + ["[SEP]"]
  return split_tokens
def get_input_ids(token , tokenizer , MAX):
  segment_ids = [0] * len(token)
  input_ids = tokenizer.convert_tokens_to_ids(token)

  input_mask = [1] * (len(input_ids) -1 )
  input_mask += [0]
  
  padding = [0] * (MAX -len(input_ids) )

  input_ids += padding
  input_mask += padding
  segment_ids += padding

  assert len(input_ids) == MAX
  return input_ids 
def get_input_mask(token, tokenizer, MAX):
  
  input_ids = tokenizer.convert_tokens_to_ids(token)

  input_mask = [1]* (len(input_ids) -1 )
  input_mask += [0]

  padding = [0] * (MAX -len(input_ids) )

  input_mask += padding

  assert len(input_mask) == MAX
  return input_mask

In [32]:
input_ids

tensor([   2, 1649, 4191, 4146,   22,    3,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])

In [52]:
model.eval()
sentence = input()
split_token = split_token_get(sentence,tokenizer,128)
input_ids = get_input_ids(split_token,tokenizer,128)
input_mask = get_input_mask(split_token,tokenizer,128)

input_ids = [input_ids]
input_mask = [input_mask]

input_ids = torch.tensor(input_ids)
input_mask = torch.tensor(input_mask)

outputs = model(input_ids = input_ids,
                attention_mask = input_mask)
logits = outputs[0]
logits = logits.detach().cpu().numpy()
probit = softmax(logits)[0][1]
print(probit)

첼린저
